In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/marco/anaconda3/bin/python
3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)


In [2]:
sc.stop()

In [3]:
#sc.stop()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession\
    .builder\
    .master("local[4]")\
    .appName("Feature Enginering mt")\
    .getOrCreate()

sc = spark.sparkContext

In [4]:
sc._conf.get('spark.driver.memory')

'16g'

In [5]:
sc._conf.getAll()

[('spark.app.name', 'Feature Enginering mt'),
 ('spark.app.id', 'local-1638931493656'),
 ('spark.master', 'local[4]'),
 ('spark.local.dir', '/home/marco/claseBigData/ProyectoBD/tmp'),
 ('spark.app.startTime', '1638931493558'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.driver.host', '192.168.3.5'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '38355'),
 ('spark.ui.showConsoleProgress', 'true')]

In [6]:
sc.version

'3.0.3'

In [7]:
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/marco/anaconda3/bin/python
3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)


![imagenes/](img/DataPreparation.png)

#### Carga de fuentes de información

Cabeceras:
    - origen: numero de origen,
    - destino: numero de destino,
    - tipo: tipo de llamada,
    - segundos: duración de llamada (seg),
    - llamadas: numero de llmadas, 
    - mensajes: numero de sms

In [8]:
from pyspark.sql.types import StructField, StructType, StringType,DoubleType
custom_schema = StructType([
    StructField("origen", StringType(), True),
    StructField("destino", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("segundos", DoubleType(), True),
    StructField("llamadas", DoubleType(), True),
    StructField("mensajes", DoubleType(), True)
])

In [9]:
mtDf = spark.read.format("csv")\
        .schema(custom_schema)\
        .option("header","false")\
        .load("../rawData/mt/week2/20150310.csv.sanitaized")

In [10]:
mtDf.show(truncate=True)

+--------------------+--------------------+-------+--------+--------+--------+
|              origen|             destino|   tipo|segundos|llamadas|mensajes|
+--------------------+--------------------+-------+--------+--------+--------+
|0917f9c3329fcdbee...|                 555|SERVICE|     0.0|     0.0|     1.0|
|557cc43b17fcee524...|c693af5d121c68d63...| MOBILE|    47.0|     1.0|     1.0|
|332e59a9c83d6b690...|dec6d3468db94542f...| MOBILE|     0.0|     0.0|     1.0|
|b542cacb488db4d18...|ea95e0ca4d591b2d4...| MOBILE|     0.0|     0.0|     1.0|
|87c596657693f26f5...|6da01f1adeb493ffb...| MOBILE|    23.0|     1.0|     0.0|
|a3549a1533368dcfb...|8c904e4628bf3e78b...| MOBILE|    60.0|     2.0|     0.0|
|44c49f8c6cfaa0d94...|                 555|SERVICE|     0.0|     0.0|     1.0|
|03efe1e8f5c45e5de...|748dad3c8816b5051...| MOBILE|   180.0|     1.0|     0.0|
|696e2cd1f2874f0ec...|02782fd0428887eda...| MOBILE|    71.0|     4.0|     0.0|
|01945c61aa576fc3d...|d8b0e1c822bc65036...| MOBILE| 

moDf = spark.read.format("csv")\
        .schema(custom_schema)\
        .option("header","false")\
        .load("../rawData/mo/week2/20150310.csv")

In [11]:
mtDf.show()

+--------------------+--------------------+-------+--------+--------+--------+
|              origen|             destino|   tipo|segundos|llamadas|mensajes|
+--------------------+--------------------+-------+--------+--------+--------+
|0917f9c3329fcdbee...|                 555|SERVICE|     0.0|     0.0|     1.0|
|557cc43b17fcee524...|c693af5d121c68d63...| MOBILE|    47.0|     1.0|     1.0|
|332e59a9c83d6b690...|dec6d3468db94542f...| MOBILE|     0.0|     0.0|     1.0|
|b542cacb488db4d18...|ea95e0ca4d591b2d4...| MOBILE|     0.0|     0.0|     1.0|
|87c596657693f26f5...|6da01f1adeb493ffb...| MOBILE|    23.0|     1.0|     0.0|
|a3549a1533368dcfb...|8c904e4628bf3e78b...| MOBILE|    60.0|     2.0|     0.0|
|44c49f8c6cfaa0d94...|                 555|SERVICE|     0.0|     0.0|     1.0|
|03efe1e8f5c45e5de...|748dad3c8816b5051...| MOBILE|   180.0|     1.0|     0.0|
|696e2cd1f2874f0ec...|02782fd0428887eda...| MOBILE|    71.0|     4.0|     0.0|
|01945c61aa576fc3d...|d8b0e1c822bc65036...| MOBILE| 

In [12]:
#moDf.filter(moDf.tipo!="MOBILE").count()   # 994,258

In [13]:
#moDf.filter(moDf.tipo=="MOBILE").count()   # 7,151,552

In [14]:
from pyspark.sql import functions as F

In [15]:
#moDf.groupBy("origen").agg(F.sum("segundos").alias("sum_segundos"), \
 #                          F.countDistinct("destino").alias("destinos"), \
  #                         F.sum("llamadas").alias("sum_llamadas"), \
   #                        F.sum("mensajes").alias("sum_sms")).show()

In [16]:
mtDf.groupBy("tipo").agg(F.count("*"),F.countDistinct(mtDf.origen)).show()

+-------------+--------+-------------+
|         tipo|count(1)|count(origen)|
+-------------+--------+-------------+
|     LANDLINE|  275689|       238683|
|       OTHERS|     774|          754|
|      SERVICE| 2292564|      1846418|
|       MOBILE|10405254|      3164210|
|INTERNATIONAL|  115482|        98387|
+-------------+--------+-------------+



In [17]:
#moDf.groupBy("origen","tipo").agg(F.when(F.col("tipo")=="MOBILE",F.sum("segundos")).otherwise(0).alias("sum_segundos")\
#                          ).show()

In [18]:
mtDfInter01=mtDf\
    .withColumn("segMobile", F.when(mtDf.tipo == "MOBILE", mtDf.segundos).otherwise(0))\
    .withColumn("segService", F.when(mtDf.tipo == "SERVICE", mtDf.segundos).otherwise(0))\
    .withColumn("segInternational", F.when(mtDf.tipo == "INTERNATIONAL", mtDf.segundos).otherwise(0))\
    .withColumn("segOthers", F.when(~((mtDf.tipo=="INTERNATIONAL")|\
                                            (mtDf.tipo=="SERVICE")|\
                                            (mtDf.tipo=="MOBILE")), mtDf.segundos).otherwise(0))\
    .withColumn("llaMobile", F.when(mtDf.tipo == "MOBILE", mtDf.llamadas).otherwise(0))\
    .withColumn("llaService", F.when(mtDf.tipo == "SERVICE", mtDf.llamadas).otherwise(0))\
    .withColumn("llaInternational", F.when(mtDf.tipo == "INTERNATIONAL", mtDf.llamadas).otherwise(0))\
    .withColumn("llaOthers", F.when(~((mtDf.tipo=="INTERNATIONAL")|\
                                            (mtDf.tipo=="SERVICE")|\
                                            (mtDf.tipo=="MOBILE")), mtDf.llamadas).otherwise(0))\
    .withColumn("smsMobile", F.when(mtDf.tipo == "MOBILE", mtDf.mensajes).otherwise(0))\
    .withColumn("smsService", F.when(mtDf.tipo == "SERVICE", mtDf.mensajes).otherwise(0))\
    .withColumn("smsInternational", F.when(mtDf.tipo == "INTERNATIONAL", mtDf.mensajes).otherwise(0))\
    .withColumn("smsOthers", F.when(~((mtDf.tipo=="INTERNATIONAL")|\
                                            (mtDf.tipo=="SERVICE")|\
                                            (mtDf.tipo=="MOBILE")), mtDf.mensajes).otherwise(0))\
    .withColumn("desMobile", F.when(mtDf.tipo == "MOBILE", mtDf.destino))\
    .withColumn("desService", F.when(mtDf.tipo == "SERVICE", mtDf.destino))\
    .withColumn("desInternational", F.when(mtDf.tipo == "INTERNATIONAL", mtDf.destino))\
    .withColumn("desOthers", F.when(~((mtDf.tipo=="INTERNATIONAL")|\
                                            (mtDf.tipo=="SERVICE")|\
                                            (mtDf.tipo=="MOBILE")), mtDf.destino))

In [19]:
mtDfInter01.limit(10).toPandas().head()

,origen,destino,tipo,segundos,llamadas,mensajes,segMobile,segService,segInternational,segOthers,...,llaInternational,llaOthers,smsMobile,smsService,smsInternational,smsOthers,desMobile,desService,desInternational,desOthers
0,0917f9c3329fcdbee8265972cb788574d1a56ec779dff5...,555,SERVICE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,None,555,None,None
1,557cc43b17fcee5245984b532984c25993adfe852e4bb5...,c693af5d121c68d63c589ea557dd732a999a0cc5c5e12d...,MOBILE,47.0,1.0,1.0,47.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,c693af5d121c68d63c589ea557dd732a999a0cc5c5e12d...,None,None,None
2,332e59a9c83d6b6909a678212f79e208f6d063be5e391e...,dec6d3468db94542f075899818949c4dcb9b39e8238774...,MOBILE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,dec6d3468db94542f075899818949c4dcb9b39e8238774...,None,None,None
3,b542cacb488db4d18704f4c409f8257b5115c39bf7d28f...,ea95e0ca4d591b2d40d9b3a9e404b859c35da9c7f40c78...,MOBILE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,ea95e0ca4d591b2d40d9b3a9e404b859c35da9c7f40c78...,None,None,None
4,87c596657693f26f57f14eeabb03f5bd8944308029d2af...,6da01f1adeb493ffb38115a7912792946b29faa74709b4...,MOBILE,23.0,1.0,0.0,23.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6da01f1adeb493ffb38115a7912792946b29faa74709b4...,None,None,None


In [20]:
mtDfInter01.count()  #8,145,810  #13,089,763

13089763

In [21]:
mtDfInter01 = mtDfInter01.groupBy("origen")\
            .agg(\
                 ### Agregando en Segundos
                 F.sum("segundos").alias("sumSeg")\
                ,F.sum("segMobile").alias("sumSegMobile")\
                ,F.sum("segService").alias("sumSegService")\
                ,F.sum("segInternational").alias("sumSegInternational")\
                ,F.sum("segOthers").alias("sumSegOthers")\
                 ### Agregando en llamadas
                ,F.sum("llamadas").alias("sumLla")\
                ,F.sum("llaMobile").alias("sumLlaMobile")\
                ,F.sum("llaService").alias("sumLlaService")\
                ,F.sum("llaInternational").alias("sumLlaInternational")\
                ,F.sum("llaOthers").alias("sumLlaOthers")\
                  ### Agregando en mensajes
                ,F.sum("mensajes").alias("sumSms")\
                ,F.sum("smsMobile").alias("sumSmsMobile")\
                ,F.sum("smsService").alias("sumSmsService")\
                ,F.sum("smsInternational").alias("sumSmsInternational")\
                ,F.sum("smsOthers").alias("sumSmsOthers")\
                 ### Agregando destinos distintos
                ,F.countDistinct("destino").alias("nroDest")\
                ,F.countDistinct("desMobile").alias("nroDestMobile")\
                ,F.countDistinct("desService").alias("nroDestService")\
                ,F.countDistinct("desInternational").alias("nroDestInternational")\
                ,F.countDistinct("desOthers").alias("nroDestOthers")\
                )

In [22]:
mtDfInter01.count()  # 2,210,592 # 3,578,079

3578079

In [23]:
mtDfInter01.limit(10).toPandas().head()

,origen,sumSeg,sumSegMobile,sumSegService,sumSegInternational,sumSegOthers,sumLla,sumLlaMobile,sumLlaService,sumLlaInternational,...,sumSms,sumSmsMobile,sumSmsService,sumSmsInternational,sumSmsOthers,nroDest,nroDestMobile,nroDestService,nroDestInternational,nroDestOthers
0,ae2053561faf605eda4f240e1f3ed8b06290b0dfdf33c3...,585.0,585.0,0.0,0.0,0.0,11.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,9,0,0,0
1,c42b280c21670fb04496bfa9c5f026a71474f4edc9caf8...,121.0,121.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,6.0,6.0,0.0,0.0,0.0,4,4,0,0,0
2,1921ccc15bc21eb383894a86e8b881fffa1a6f0e353782...,102.0,102.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,...,16.0,16.0,0.0,0.0,0.0,8,8,0,0,0
3,eb4c3d07e6f98ad05ce579a95f6aa19022483909ede73c...,560.0,560.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,5,0,0,0
4,3fe43e212755835a01c0d1999d828809749f4e9d2c9e45...,619.0,619.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,...,3.0,0.0,3.0,0.0,0.0,5,3,2,0,0


In [24]:
#moDf.filter(moDf.origen=="5f96f009c808466497ce0a5e19126cd9144be71c3245766948a57a2e").show()

In [25]:
mtDf.filter(mtDf.origen=="5f96f009c808466497ce0a5e19126cd9144be71c3245766948a57a2e").show()

+--------------------+--------------------+-------------+--------+--------+--------+
|              origen|             destino|         tipo|segundos|llamadas|mensajes|
+--------------------+--------------------+-------------+--------+--------+--------+
|5f96f009c80846649...|f567a1dfa8f855c75...|       MOBILE|    15.0|     1.0|     0.0|
|5f96f009c80846649...|919d4f8ff96c308ea...|       MOBILE|    18.0|     1.0|     0.0|
|5f96f009c80846649...|                 123|      SERVICE|     0.0|     0.0|     2.0|
|5f96f009c80846649...|d3d860c3858753962...|INTERNATIONAL|   109.0|     4.0|     0.0|
|5f96f009c80846649...|536f006f14cd4b900...|INTERNATIONAL|  1571.0|     2.0|     2.0|
|5f96f009c80846649...|b912396f6cc122e4e...|       MOBILE|     0.0|     0.0|     2.0|
|5f96f009c80846649...|e82b79b2764e8fd55...|       MOBILE|     0.0|     0.0|     9.0|
+--------------------+--------------------+-------------+--------+--------+--------+



In [26]:
### Agregando Proporciones
### Agregando proporciones en segundos con un artificio para evitar la división por 0
### Agregando proporciones en llamadas con un artificio para evitar la división por 0
### Agregando proporciones en mensajes con un artificio para evitar la división por 0
### Agregando proporciones en nro de destinos con un artificio para evitar la división por 0
mtDfInter01 = mtDfInter01\
    .withColumn("PropSegMobile", F.round((mtDfInter01.sumSegMobile+0.5)/(mtDfInter01.sumSeg+1),5))\
    .withColumn("PropSegService", F.round((mtDfInter01.sumSegService+0.5)/(mtDfInter01.sumSeg+1),5))\
    .withColumn("PropSegInternational", F.round((mtDfInter01.sumSegInternational+0.5)/(mtDfInter01.sumSeg+1),5))\
    .withColumn("PropLlaMobile", F.round((mtDfInter01.sumLlaMobile+0.5)/(mtDfInter01.sumLla+1),5))\
    .withColumn("PropLlaService", F.round((mtDfInter01.sumLlaService+0.5)/(mtDfInter01.sumLla+1),5))\
    .withColumn("PropLlaInternational", F.round((mtDfInter01.sumLlaInternational+0.5)/(mtDfInter01.sumLla+1),5))\
    .withColumn("PropSmsMobile", F.round((mtDfInter01.sumSmsMobile+0.5)/(mtDfInter01.sumLla+1),5))\
    .withColumn("PropSmsService", F.round((mtDfInter01.sumSmsService+0.5)/(mtDfInter01.sumLla+1),5))\
    .withColumn("PropSmsInternational", F.round((mtDfInter01.sumSmsInternational+0.5)/(mtDfInter01.sumSms+1),5))\
    .withColumn("PropNroDestMobile", F.round((mtDfInter01.nroDestMobile+0.5)/(mtDfInter01.nroDest+1),5))\
    .withColumn("PropNroDestService", F.round((mtDfInter01.nroDestService+0.5)/(mtDfInter01.nroDest+1),5))\
    .withColumn("PropNroDestInternational", F.round((mtDfInter01.nroDestInternational+0.5)/(mtDfInter01.nroDest+1),5))

In [27]:
mtDfInter01.limit(10).toPandas().head()

,origen,sumSeg,sumSegMobile,sumSegService,sumSegInternational,sumSegOthers,sumLla,sumLlaMobile,sumLlaService,sumLlaInternational,...,PropSegInternational,PropLlaMobile,PropLlaService,PropLlaInternational,PropSmsMobile,PropSmsService,PropSmsInternational,PropNroDestMobile,PropNroDestService,PropNroDestInternational
0,ae2053561faf605eda4f240e1f3ed8b06290b0dfdf33c3...,585.0,585.0,0.0,0.0,0.0,11.0,11.0,0.0,0.0,...,0.00085,0.95833,0.04167,0.04167,0.04167,0.04167,0.50000,0.95000,0.05000,0.05000
1,c42b280c21670fb04496bfa9c5f026a71474f4edc9caf8...,121.0,121.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,0.00410,0.90000,0.10000,0.10000,1.30000,0.10000,0.07143,0.90000,0.10000,0.10000
2,1921ccc15bc21eb383894a86e8b881fffa1a6f0e353782...,102.0,102.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,...,0.00485,0.87500,0.12500,0.12500,4.12500,0.12500,0.02941,0.94444,0.05556,0.05556
3,eb4c3d07e6f98ad05ce579a95f6aa19022483909ede73c...,560.0,560.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.00089,0.91667,0.08333,0.08333,0.08333,0.08333,0.50000,0.91667,0.08333,0.08333
4,3fe43e212755835a01c0d1999d828809749f4e9d2c9e45...,619.0,619.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,...,0.00081,0.95000,0.05000,0.05000,0.05000,0.35000,0.12500,0.58333,0.41667,0.08333


In [28]:
mtDfInter01.count()  #2,210,592 #3,578,079

3578079

In [29]:
mtDfInter01.coalesce(5).write.format("parquet").mode("overwrite").save("../prepData/mt_variables_iniciales.parquet")

### Contactos distintos

In [12]:
mtDf.select("origen","destino").coalesce(5).write.format("parquet").mode("overwrite").save("../prepData/mt_destinos.parquet")